# Морфология 4
В этом ноутбуке описана подготовка данных для задачи POS-tagging. А также пара простых моделей на keras, решающих данную задачу. Оригинальная задача и ноутбук есть в контесте: https://www.kaggle.com/c/rupos2018/overview

## Часть 1. Загрузка корпуса
Здесь мы прочитаем корпуса из csv и разложим их по спискам.

In [1]:
# для совместимости со вторым питоном
from __future__ import print_function
import io

In [2]:
# Имена файлов с данными.
TRAIN_FILENAME = "data/input/train.csv"
TEST_FILENAME = "data/input/test.csv"

In [3]:
# Считывание файлов.
from collections import namedtuple
WordForm = namedtuple("WordForm", "word pos gram")

def get_sentences(filename, is_train):
    sentences = []
    with io.open(filename, "r", encoding='utf-8') as r:
        # Пропускаем заголовок
        next(r)
        sentence = [] # будем заполнять список предложений
        for line in r:
            # предложения отделены по '\n'
            if len(line.strip()) == 0:
                if len(sentence) == 0:
                    continue
                sentences.append(sentence)
                sentence = []
                continue
            if is_train:
                # Формат: индекс\tномер_в_предложении\tсловоформа\tPOS#Грамемы
                word = line.strip().split("\t")[2]
                pos = line.strip().split("\t")[3].split("#")[0]
                gram = line.strip().split("\t")[3].split("#")[1]
                sentence.append(WordForm(word, pos, gram))
            else:
                word = line.strip().split("\t")[2]
                sentence.append(word)
        if len(sentence) != 0:
            sentences.append(sentence)
    return sentences

In [4]:
train = get_sentences(TRAIN_FILENAME, True)
test = get_sentences(TEST_FILENAME, False)

In [5]:
# Выыедем, что получилось
for wordform in train[0][:10]:
    print(wordform.word, '\t', wordform.pos, '\t', wordform.gram)

А 	 CONJ 	 _
ведь 	 PART 	 _
для 	 ADP 	 _
конкретных 	 ADJ 	 Case=Gen|Degree=Pos|Number=Plur
изделий 	 NOUN 	 Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur
зачастую 	 ADV 	 Degree=Pos
нужен 	 ADJ 	 Degree=Pos|Gender=Masc|Number=Sing|Variant=Brev
монокристалл 	 NOUN 	 Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
не 	 PART 	 _
только 	 PART 	 _


Для простоты далее будем использовать токены слов и POS-теги. Но чтобы определять грамматические значения нужно еще провести некоторые манипуляции с данными, описанные в оригинальном ноутубке. Мы же ограничимся только определением частей речи

## Часть 2. Подготовка эмбеддингов

Обычно в качестве признаков для обучения сеток используются словные эмбеддинги. Для этого можно скачать предобученные и сохранить их в матрицу, где в расположатся векторы эмбеддингах по индексам, соответсвующих слов

In [6]:
#запомним все уникальные слова и POS-теги в корпусе
word_set = set()
pos_set = set()
for sent in train:
    for wordform in sent:
        word_set.add(wordform.word.lower())
        pos_set.add(wordform.pos)

In [7]:
for word in list(word_set)[:10]: 
    print(word, end=', ')
print()
print(pos_set)

лордом, барсук, бракосочетании, измерили, неутешительный, 157, снабжает, госструктур, метаболизме, сваливались, 
{'ADP', 'SCONJ', 'ADJ', 'CONJ', 'AUX', 'INTJ', 'PART', 'ADV', 'SYM', 'VERB', 'NUM', 'X', 'DET', 'PRON', 'NOUN', 'PUNCT', 'PROPN'}


In [23]:
#Загрузите эмбеддинги c https://nlp.stanford.edu/projects/glove/ или другие, которые вам нравятся и пропишите путь к ним

# https://wikipedia2vec.github.io/wikipedia2vec/pretrained/
import numpy as np

word_embeddings_path = 'data/ruwiki_20180420_100d.txt'
word2idx = {}
word_embeddings = []
embedding_size = None
#Загружаем эмбеддинги
with io.open(word_embeddings_path, 'r', encoding="utf-8") as f_em:
    for line in f_em:
        split = line.strip().split(" ")
        # Совсем короткие строки пропускаем
        if len(split) <= 2:
            continue
        # Встретив первую подходящую строку, фиксируем размер эмбеддингов
        if embedding_size is None:
            embedding_size = len(split) - 1
            # Также нициализируем эмбеддинги для паддингов и неизвестных слов
            word2idx["PADDING_TOKEN"] = len(word2idx)
            word_embeddings.append(np.zeros(embedding_size))

            word2idx["UNKNOWN_TOKEN"] = len(word2idx)
            word_embeddings.append(np.random.uniform(-0.25, 0.25, embedding_size))
        # После этого все эмбеддинги должны быть одинаковой длины
        if len(split) - 1 != embedding_size:
            continue
            
        #Если слова нет в корпусе, то не будем для него запоминать эмбеддинг        
        if (split[0] not in word_set):
            continue
        
        word_embeddings.append(np.asarray(split[1:], dtype='float32'))
        word2idx[split[0]] = len(word2idx)

word_embeddings = np.array(word_embeddings, dtype='float32')

In [24]:
len(word_set & set(word2idx.keys()))

91330

In [25]:
len(word_set)

98880

In [26]:
word2idx_set = set(word2idx.keys())

In [28]:
for word in word_set:
    if not word in word2idx_set:
        word2idx[word] = len(word2idx)
        word2idx_set.add(word)

In [29]:
len(word_set & set(word2idx.keys()))

98880

Как-то эмбеддинги не сильно подходят для данного корпуса поэтому, просто инициализируем рандмно матрицу эмбеддингов при определении сетки. Вам же предлагается все-таки поискать подходящие эмбеддинги и использовать их при обучении.

## Часть 3. Подготовка данных
Теперь нам остается только пронумеровать все слова и POS-теги и можно переходить к обучению сеток.

In [31]:
word_to_index = {'PAD' : 0, 'UNK' : 1}
for word in word_set:
    word_to_index[word] = len(word_to_index)

In [32]:
pos_to_index = {"PAD": 0}
index_to_pos = {0: "PAD"}
for pos in pos_set:
    pos_to_index[pos] = len(pos_to_index)
    index_to_pos[len(index_to_pos)] = pos

In [33]:
# для полносвязной сетки просто захреначим все в один список
data_X = []
data_Y = []
for sent in train:
    for wordform in sent:
        data_X.append(word_to_index[wordform.word.lower()])
        data_Y.append(pos_to_index[wordform.pos])

In [34]:
print(data_X[:10])
print(data_Y[:10])

[30242, 80873, 13379, 60856, 61168, 54174, 48329, 57079, 44875, 32592]
[4, 7, 1, 3, 15, 8, 3, 15, 7, 7]


## Часть 4. Полносвязная сеть
Самой простой моделью является обычный перцептрон. На вход сетки будем подавать просто эмдеддинг каждого слова, на выходе ожидать распредедение вероятностей по тегам. В качестве фреймворка достаточно будет использовать keras и его Sequential модель (https://keras.io/models/sequential/), в которую слои добавляются последовательно, с помощью метода `add`.

In [35]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Activation, Flatten

Using TensorFlow backend.


In [36]:
model = Sequential()
# на самом деле на вход сетки будет добавляться индекс слова, а слой эмбеддинга будет возвращать для него вектор
model.add(Embedding(input_length=1, input_dim=len(word_to_index), output_dim=50, embeddings_initializer='random_uniform',
                    trainable=False)) # матрицу эмбеддингов просто инициализируем нормальным распределением и отключим обучение
# далее нам нужно схлопнуть трехмерный тензор с одной фиктивной размерностью в двумерный
model.add(Flatten())
model.add(Dense(100)) # основной полносвязный слой
model.add(Activation('relu')) # для приличия добавим функцию активации
model.add(Dense(len(pos_to_index))) # выходной слой тоже полносвязный размерности по кол-ву тегов
model.add(Activation('softmax')) # ну и в конце делаем softmax, чтобы получить распределение
model.summary() # вывод получившейся модели

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 50)             4944100   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 18)                1818      
_________________________________________________________________
activation_2 (Activation)    (None, 18)                0         
Total params: 4,951,018
Trainable params: 6,918
Non-trainable params: 4,944,100
________________________________________

In [37]:
# компилируем модель
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
# и обучаем
model.fit(np.array(data_X), np.array(data_Y), epochs=1, batch_size=256)

Epoch 1/1
850689/850689 [==============================] - 4s 4us/step - loss: 1.4388 - accuracy: 0.5395


Проверка обученности модели остается за вами. Этот пример лишь для того, чтобы показать как собрать сетку и скормить ей данные.

## Часть 5. Рекуррентая сеть.

Далее рассмотрим более приближенную к SOTA модель. Ей является рекуррентая сеть, которая принимает эмбеддинги слов в предложении и генерирует для них распределение вероятностей. Основным отличием от прошлой в том, что теперь мы будем использовать соседние слова как раз за счет рекуррентого слоя. Для этой модели мы уже будем использовать функциональный способ задания модели все того же кераса (https://keras.io/models/model/).

In [57]:
word_embeddings

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.14992113, -0.1772393 ,  0.10720362, ..., -0.1589421 ,
        -0.24490474, -0.02399684],
       [ 0.1216    , -0.3251    , -0.1264    , ...,  0.2268    ,
        -0.0277    , -0.0951    ],
       ...,
       [ 0.2397    , -0.0969    , -0.1736    , ...,  0.2676    ,
        -0.1899    , -0.4815    ],
       [ 0.301     ,  0.0516    , -0.0025    , ...,  0.1854    ,
        -0.04      , -0.003     ],
       [ 0.6093    ,  0.2419    ,  0.0996    , ...,  0.4635    ,
        -0.1319    , -0.3728    ]], dtype=float32)

In [58]:
import numpy as np

embedding_matrix = np.zeros((len(word_set) + 2, 100))
for word, i in word2idx.items():
    if i < len(word_embeddings):
        embedding_matrix[i] = word_embeddings[i]
    else:
        embedding_matrix[i] = np.random.uniform(size=100)

In [51]:
from keras.layers import LSTM, TimeDistributed,Bidirectional, Input
from keras.models import Model

In [59]:
# В начале задается входной слой, в котором мы укажем входную размерность. 
# Это будет None, т.к. мы заранее не знаем, какой будет длина каждого предложения 
input_layer = Input(shape=(None,), name='input')
# Далее идет все тот же слой эмеддинга, которому мы на вход подаем предыдущий слой (в этом и суть functional APO)
embeddings_layer = Embedding(input_dim=len(word_to_index), output_dim=100, 
                             trainable=False, weights=[embedding_matrix],
                             name='embedding')(input_layer)
# Итак, основным слоем здесь будет двусторонний LSTM, который будет возвращать вектор для каждого слова (return_sequences=True) 
blstm_layer = Bidirectional(LSTM(200, return_sequences=True), name='blstm')(embeddings_layer)
# Аналогично т.к. у нас здесь вектора для каждого слоя, то и полносвязный слой должен применяться для каждого слоя 
# по-отдельности. Поэтому полносвязный слой оборачивается в  TimeDistributed
result_layer = TimeDistributed(Dense(len(pos_to_index), activation='softmax', name='result'))(blstm_layer)
# собственно определяем модель входными и выходными слоями
model = Model(inputs=[input_layer], outputs=result_layer)
# компилируем
model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])
# выводим архитектуру
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         9888200   
_________________________________________________________________
blstm (Bidirectional)        (None, None, 400)         481600    
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 18)          7218      
Total params: 10,377,018
Trainable params: 488,818
Non-trainable params: 9,888,200
_________________________________________________________________


Далее нам нужно было бы распределить слова по предложениям, распределить по группам по длине, выравнить предложения по длине в одной групе, заполнив недостающие слова паддингами. Но это довольно неприятный процесс, а мне просто хочется запустить сетку и проверить, что она вообще работает, что сошлись все разверности. Поэтому просто раскидаем по 10 слов с помощью `numpy.reshape`

In [60]:
# rnnX = np.reshape(data_X[:850000], (-1,10))
# rnnY = np.reshape(data_Y[:850000], (-1,10,1))

In [61]:
# np.shape(rnnX)

Ну и проверим, что оно вообще работает.

In [62]:
#model.fit(rnnX, rnnY, epochs=1, batch_size=256)

## Часть 6. Задание
В качестве упражения предлагается довести до ума обучения второй модели: распределить слова по предложениям, написать тестирование модели и собственно посмотреть как оно обучилось. Тестировать предлагаю на последней 1000 предложений, обучать - на остальном. Кто уверен в своих желаниях, то может решить оригинальную задачу: предсказывать также грамматические категории. А мы же перейдем на следующем семинаре к более приятному фреймворку - PyTorch. 

In [63]:
# data_X = []
# data_Y = []
# for sent in train:
#     data_X.append([word_to_index[wordform.word.lower()] for wordform in sent])
#     data_Y.append([pos_to_index[wordform.pos] for wordform in sent])
data_X = []
data_Y = []
for sent in train:
    data_X.append([word2idx[wordform.word.lower()] for wordform in sent])
    data_Y.append([pos_to_index[wordform.pos] for wordform in sent])

In [64]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y, test_size = 1000)

# X_train, y_train = zip(*sorted(zip(X_train, y_train), key=lambda item: len(item[0])))
X_train = np.array(X_train)
y_train = np.array(y_train)

In [65]:
print(len(X_test), len(y_test))

1000 1000


In [66]:
def get_batch_with_alignment(X, y):
    batch_size = len(X)
    max_sent_len = len(max(X, key=lambda item: len(item)))
    X_batch = np.array([sent + [word2idx["PADDING_TOKEN"]] * (max_sent_len - len(sent)) for sent in X])
    pos_tags = np.array([sent + [pos_to_index["PAD"]] * (max_sent_len - len(sent)) for sent in y])
    
    y_batch = np.zeros((len(X), max_sent_len, len(pos_to_index)))
    for i, row in enumerate(pos_tags):
        for j, pos in enumerate(row):
            y_batch[i, j, pos] = 1
    return (X_batch, y_batch)

In [67]:
batch_size = 128
batches = []
idx = 0
while idx + batch_size < len(X_train):
    idxs = np.arange(idx, idx + batch_size)
    idx += batch_size
    np.random.shuffle(idxs)
    X = X_train[idxs]
    y = y_train[idxs]
    
    batches.append(get_batch_with_alignment(X, y))

In [68]:
import tqdm

for batch in tqdm.tqdm(batches):
    model.train_on_batch(batch[0], batch[1])

100%|██████████| 368/368 [02:18<00:00,  2.60it/s]


In [69]:
batch = get_batch_with_alignment(X_test, y_test)
res = model.evaluate(batch[0], batch[1])

for name, val in zip(model.metrics_names, res):
    print(name, val)

1000/1000 [==============================] - 2s 2ms/step
loss 0.09356270033121108
accuracy 0.9724166393280029
